In [1]:
import os
from os.path import join, basename
DATASETS = '/data/'

import glob
from tqdm import tqdm

import cv2
import numpy as np
from basicsr.metrics import calculate_psnr, calculate_ssim
from imresize import imresize, convertDouble2Byte
from skimage import color

import matplotlib.pyplot as plt

crop_border = 0

In [2]:
gt_images = glob.glob(join(DATASETS, 'ISTD_adjusted/test/shadow_free/*.png'))
gt_images = sorted(gt_images, key=lambda x: (int(basename(x).split('-')[0]), int(basename(x).split('-')[1][:-4])))

gt_masks = glob.glob(join(DATASETS, 'ISTD_adjusted/test/mask/*.png'))
gt_masks = sorted(gt_masks, key=lambda x: (int(basename(x).split('-')[0]), int(basename(x).split('-')[1][:-4])))

output_images = glob.glob(join(DATASETS, 'ISTD_adjusted/results/*.png'))
output_images = sorted(output_images, key=lambda x: (int(basename(x).split('-')[0]), int(basename(x).split('-')[1][:-4])))

print("number of images for gt_images, gt_masks, output_images", len(gt_images), len(gt_masks), len(output_images))

number of images for gt_images, gt_masks, output_images 540 540 540


In [3]:
s_psnr_all = []
s_ssim_all = []

ns_psnr_all = []
ns_ssim_all = []

a_psnr_all = []
a_ssim_all = []

rl = []
ra = []
rb = []

total_dist = 0
total_pixel = 0
total_dists = 0
total_pixels = 0
total_distn = 0
total_pixeln = 0

for gt_path, output_path, mask_path in tqdm(zip(gt_images, output_images, gt_masks)):
    gt_img = cv2.imread(gt_path)[..., ::-1].astype(np.float64)
    output_img = cv2.imread(output_path)[..., ::-1].astype(np.float64)
    mask =  cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
    if len(mask.shape) > 2:
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask = mask.astype(np.float64)

    gt_img = np.clip(imresize(gt_img, output_shape=(256, 256)) / 255., 0.0, 1.0)
    output_img = np.clip(imresize(output_img, output_shape=(256, 256)) / 255., 0.0, 1.0)
    mask = np.clip(imresize(mask,  output_shape=(256, 256)) / 255., 0.0, 1.0)

    lab_gt_img = cv2.cvtColor(np.float32(gt_img), cv2.COLOR_RGB2LAB)
    lab_output_img = cv2.cvtColor(np.float32(output_img), cv2.COLOR_RGB2LAB)

    gt_img = convertDouble2Byte(gt_img)
    output_img = convertDouble2Byte(output_img)

    s_mask = mask.copy()
    ns_mask = 1 - s_mask.copy()

    s_gt_img = gt_img * s_mask[..., None].repeat(3, 2)
    s_output_img = output_img * s_mask[..., None].repeat(3, 2)
    s_psnr_all.append(calculate_psnr(s_gt_img, s_output_img, crop_border=crop_border, input_order='HWC'))
    s_ssim_all.append(calculate_ssim(s_gt_img, s_output_img, crop_border=crop_border, input_order='HWC'))
    
    ns_gt_img = gt_img * ns_mask[..., None].repeat(3, 2)
    ns_output_img = output_img * ns_mask[..., None].repeat(3, 2)
    ns_psnr_all.append(calculate_psnr(ns_gt_img, ns_output_iPmg, crop_border=crop_border, input_order='HWC'))
    ns_ssim_all.append(calculate_ssim(ns_gt_img, ns_output_img, crop_border=crop_border, input_order='HWC'))

    a_psnr_all.append(calculate_psnr(gt_img, output_img, crop_border=crop_border, input_order='HWC'))
    a_ssim_all.append(calculate_ssim(gt_img, output_img, crop_border=crop_border, input_order='HWC'))

    dist = np.abs(lab_gt_img - lab_output_img)
    total_dist = total_dist + np.sum(dist)
    total_pixel = total_pixel + np.sum(np.ones_like(mask))

    dist = np.abs(lab_gt_img - lab_output_img) * s_mask[..., None].repeat(3, 2)
    total_dists = total_dists + np.sum(dist)
    total_pixels = total_pixels + np.sum(s_mask)

    dist = np.abs(lab_gt_img - lab_output_img) * ns_mask[..., None].repeat(3, 2)
    total_distn = total_distn + np.sum(dist)
    total_pixeln = total_pixeln + np.sum(ns_mask)

print(f'Average: ALL-RMSE: {total_dist / total_pixel:.6f}, NS-RMSE: {total_distn / total_pixeln:.6f}, S-RMSE: {total_dists / total_pixels:.6f}')
print(f'Average: ALL-PSNR: {sum(a_psnr_all) / len(a_psnr_all):.6f} dB, ALL-SSIM: {sum(a_ssim_all) / len(a_ssim_all):.6f}')
print(f'Average: NS-PSNR: {sum(ns_psnr_all) / len(ns_psnr_all):.6f} dB, NS-SSIM: {sum(ns_ssim_all) / len(ns_ssim_all):.6f}')
print(f'Average: S-PSNR: {sum(s_psnr_all) / len(s_psnr_all):.6f} dB, S-SSIM: {sum(s_ssim_all) / len(s_ssim_all):.6f}')

540it [02:18,  3.89it/s]

Average: ALL-RMSE: 3.116606, NS-RMSE: 2.682678, S-RMSE: 5.433195
Average: ALL-PSNR: 34.211980 dB, ALL-SSIM: 0.965837
Average: NS-PSNR: 37.511926 dB, NS-SSIM: 0.979230
Average: S-PSNR: 38.517537 dB, S-SSIM: 0.990810
